In [27]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import pandas as pd

In [28]:
## Load the trained model
model = load_model('./artifacts/ann_model.h5')

## Load the scaler
scaler = pickle.load(open('./artifacts/scaler.pkl', 'rb'))

## Load the encoders
onehot_encoder_geo = pickle.load(open('./artifacts/onehot_encoder_geo.pkl', 'rb'))
label_encoder_gender = pickle.load(open('./artifacts/label_encoder_gender.pkl', 'rb'))

In [29]:
## Example input data
input_data = {
    'CreditScore': 780,
    'Geography' : 'France',
    'Gender' : "Female",
    'Age' : 21,
    'Tenure' : 1,
    'Balance' : 340,
    'NumOfProducts' : 0,
    'HasCrCard' : 1,
    'IsActiveMember' : 1,
    'EstimatedSalary' : 10000
}

In [30]:
# One-hot encode 'Geography'
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

/Users/sankalp/Desktop/Machine Learning/Person will leave the bank or not/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [31]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,780,France,Female,21,1,340,0,1,1,10000


In [32]:
## Encode categorical variables
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,780,France,0,21,1,340,0,1,1,10000


In [33]:
## concatination one hot encoded 
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,780,0,21,1,340,0,1,1,10000,1.0,0.0,0.0


In [35]:
## Scale the input data
input_data_scaled = scaler.transform(input_df)
pd.DataFrame(input_data_scaled)

,0,1,2,3,4,5,6,7,8,9,10,11
0,1.33449,-1.091687,-1.708762,-1.37744,-1.210284,-2.652685,0.642595,0.968738,-1.567264,0.985604,-0.569844,-0.573694


In [36]:
#predict the output
prediction = model.predict(input_data_scaled)
prediction

1/1 [==============================] - 0s 116ms/step


array([[0.32062748]], dtype=float32)

In [37]:
## Convert the prediction to a human-readable format
if prediction > 0.5:
    result = 'Exited'
else:
    result = 'Not Exited'

In [38]:
print(f'The customer is likely to {result} the bank')

The customer is likely to Not Exited the bank
